In [1]:
# used to widen the cells 
from IPython.core.display import display, HTML
display(HTML("<style>.container { margin-left: 2.5% !important; width:95%; }</style>"))


In [2]:
from algosdk.v2client import algod
from algosdk import mnemonic
from algosdk import transaction
import os 

algod_address = "https://testnet-algorand.api.purestake.io/ps2"
algod_token = os.environ['PURESTAKE_ALGORAND_API']
headers = {
   "X-API-Key": algod_token,
}

algod_client = algod.AlgodClient(algod_token, algod_address, headers);

In [3]:
from algosdk import account 

In [8]:
# generate an account
private_key, address = account.generate_account()
print("Private key:", private_key)
print("Address:", address)
print("memo:", mnemonic.from_private_key(private_key))

Private key: AezFSrIDpntBtOZ5BkjBlFZOpn4EWV+9iyd2A7WLBNxp4lya6D2VXf6etYWCUEQf7WUZqB5D0LUYbOKjR5Y87Q==
Address: NHRFZGXIHWKV37U6WWCYEUCED7WWKGNIDZB5BNIYNTRKGR4WHTWW7XLMZI
memo: lens bless century depth equal galaxy pear trade song cactus gaze harsh original praise catalog flip program confirm because danger head carry then absorb sock


In [4]:
memo = "lens bless century depth equal galaxy pear trade song cactus gaze harsh original praise catalog flip program confirm because danger head carry then absorb sock"
private_key = mnemonic.to_private_key(memo)
address = mnemonic.to_public_key(memo)
print(address)

NHRFZGXIHWKV37U6WWCYEUCED7WWKGNIDZB5BNIYNTRKGR4WHTWW7XLMZI


In [5]:
def compile(approval_filename='approval.teal', clear_state_filename='clear_state.teal'):

    # pyteal to teal 
    with open(approval_filename, 'w') as f:
        compiled = compileTeal(approval_program(), Mode.Application)
        f.write(compiled)

    with open(clear_state_filename, 'w') as f:
        compiled = compileTeal(clear_state_program(), Mode.Application)
        f.write(compiled)

    # teal to bytecode 
    stdout, stderr = execute(["goal", "clerk", "compile", "-o", approval_filename+'.tok', approval_filename])
    if stderr != "":
        print(stderr)
        raise
    elif len(stdout) < 59:
        print("error in compile teal")
        raise

    stdout, stderr = execute(["goal", "clerk", "compile", "-o", clear_state_filename+'.tok', clear_state_filename])
    if stderr != "":
        print(stderr)
        raise
    elif len(stdout) < 59:
        print("error in compile teal")
        raise

    with open(approval_filename+'.tok', 'rb') as f:
        approval_bytes = f.read()

    with open(clear_state_filename+'.tok', 'rb') as f: 
        clear_state_bytes = f.read() 
        
    return approval_bytes, clear_state_bytes

In [6]:
# read app global state
def read_global_state(client, addr, app_id):   
    results = client.account_info(addr)
    apps_created = results['created-apps']
    global_states = []
    for app in apps_created :
        if app['id'] == app_id :
            global_states.append(app['params']['global-state'])
            # print(f"global_state for app_id {app_id}: ", app['params']['global-state'])
    return global_states 

In [7]:
import base64 
def decode_keys(s):
    for entry in s[0]:
        entry['key'] = base64.b64decode(entry['key']).decode('utf-8')
    return s

In [8]:
from pyteal import *    
    
def approval_program():     
    index_key = Bytes("index")
    queue_size_key = Bytes("size")
    
    user_queue_size = Btoi(Txn.application_args[0])
    on_creation = Seq([    
        App.globalPut(index_key, Int(0)),  
        If(user_queue_size > Int(61), Return(Int(0))),   # in any case, this should never be more than 61, no matter what the schema 
        App.globalPut(queue_size_key, user_queue_size),
        Return(Int(1))
    ])    
    
    on_optin = Return(Int(1))
    
    current_index = App.globalGet(index_key)
    on_push = Seq([
        App.globalPut(Itob(current_index), Txn.application_args[1]), 
        If(App.globalGet(queue_size_key) > current_index+Int(1),    # updating index 
              App.globalPut(index_key, current_index+Int(1)), 
              App.globalPut(index_key, Int(0))),
        Return(Int(1))
    ])

    program = Cond(    
            [Txn.application_id() == Int(0), on_creation],    
            [Txn.on_completion() == OnComplete.DeleteApplication, Return(Int(0))], 
            [Txn.on_completion() == OnComplete.UpdateApplication, Return(Int(0))],    
            [Txn.on_completion() == OnComplete.OptIn, on_optin],
            [Txn.application_args[0] == Bytes("push"), on_push],
    )    
    return program    
    
    
def clear_state_program():     
    program = Return(Int(1))
    return program    


In [9]:
from algosdk.future import transaction
from algosdk.future.transaction import StateSchema
from algosdk.future.transaction import OnComplete as onComplete

In [10]:
params = algod_client.suggested_params()
params.falt_fee = True
params.fee = 1000

In [11]:
approval_bytes, clear_state_bytes = compile()

local_schema = StateSchema(num_uints=1, num_byte_slices=1)    
global_schema = StateSchema(num_uints=5, num_byte_slices=5)   

on_complete = onComplete(0)
app_create_txn = transaction.ApplicationCreateTxn(
    address, 
    params, 
    on_complete, 
    approval_bytes, 
    clear_state_bytes, 
    global_schema, 
    local_schema,
    app_args = [(5).to_bytes(2, "big")]
)

app_create_txn_signed = app_create_txn.sign(private_key)
txid = algod_client.send_transaction(app_create_txn_signed)
txid

'24VLJ3ARCFMCBH5IEPRS7PGW7WYGSHUYFDMYKYVHW6KVVKACTLFQ'

In [17]:
app_id = algod_client.pending_transaction_info(txid)['application-index']
app_id

14967021

In [18]:
global_state = read_global_state(algod_client, address, app_id)
state = decode_keys(global_state)
state

[[{'key': 'index', 'value': {'bytes': '', 'type': 2, 'uint': 0}},
  {'key': 'size', 'value': {'bytes': '', 'type': 2, 'uint': 5}}]]

In [19]:
optin_tx = transaction.ApplicationOptInTxn(address, params, app_id)
algod_client.send_transaction(optin_tx.sign(private_key))

'S3XNJEXKXNQAN32WZOGBTVHTYILJQ4UYS7ZMQYL5E2MPMOCWGKSQ'

In [20]:
s = "firstcontract"
app_args = ["push".encode("utf-8"), s.encode("utf-8")]

# create unsigned transaction
call_tx = transaction.ApplicationNoOpTxn(address, params, app_id, app_args)
call_txid = algod_client.send_transaction(call_tx.sign(private_key))

AlgodHTTPError: {"message":"TransactionPool.Remember: transaction ZGZ75HFUFOPPNJ2D3HOHNFZSSEDVGAQLCHAUJVVBOIX45IGJSABA: account NHRFZGXIHWKV37U6WWCYEUCED7WWKGNIDZB5BNIYNTRKGR4WHTWW7XLMZI balance 2859000 below min 3070000 (0 assets)"}


In [140]:
params = algod_client.suggested_params()
params.falt_fee = True
params.fee = 1000